In [2]:
import pandas as pd
import numpy as np
import keras

from sklearn import metrics, cross_validation

Using TensorFlow backend.
/Users/Roman/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train = pd.read_csv("../datasets/train_without_oversampling.csv")
test = pd.read_csv("../datasets/test_without_oversampling.csv")

In [4]:
y = train.target
train.drop("target", axis=1, inplace=True)

In [5]:
train.drop("client_id", axis=1, inplace=True)
ans = test[["client_id"]]
test.drop("client_id", axis=1, inplace=True)

In [6]:
train.head()

,gender,age,credit_sum,credit_month,tariff_id,score_shk,monthly_income,credit_count,overdue_credit_count,high_edu,...,job_position_WRK,job_position_WRP,marital_status_DIV,marital_status_MAR,marital_status_UNM,marital_status_WID,city_mean_income,credit_pay,money_for_life,dif_city
0,1,48,59998.00,10,1.6,0.770249,30000.0,1.0,1.0,True,...,0,0,0,1,0,0,36905.696393,5999.800000,24000.200000,-6905.696393
1,0,28,10889.00,6,1.1,0.248514,43000.0,2.0,0.0,True,...,0,0,0,1,0,0,62130.784048,1814.833333,41185.166667,-19130.784048
2,1,32,10728.00,12,1.1,0.459589,23000.0,5.0,0.0,False,...,0,0,0,1,0,0,35838.636364,894.000000,22106.000000,-12838.636364
3,0,27,12009.09,12,1.1,0.362536,17000.0,2.0,0.0,True,...,0,0,1,0,0,0,31301.444396,1000.757500,15999.242500,-14301.444396
4,1,45,16908.89,10,1.1,0.421385,25000.0,1.0,0.0,False,...,0,0,0,1,0,0,31411.834395,1690.889000,23309.111000,-6411.834395


In [7]:
from keras.layers import Dense, Activation, Dropout
from keras.models import Sequential

In [51]:
model = Sequential()
model.add(Dense(128, input_dim=44, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_crossentropy'])

In [52]:
seed = 0
np.random.seed(seed=seed)

In [53]:
train_data, test_data, y_train, y_test = cross_validation.train_test_split(train, y, test_size = 0.3, stratify = y, \
                                                                          random_state=seed)

In [54]:
y_train = y_train.reshape((-1, 1))

/Users/Roman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':


In [ ]:
model.fit(train_data.values, y_train, nb_epoch=50, batch_size=20000)

Epoch 1/50
 40000/119522 [=========>....................] - ETA: 6s - loss: 0.8245 - binary_crossentropy: 0.8245 

In [28]:
pred = model.predict(test_data.values)
print(pred)

[[ 1.]
 [ 1.]
 [ 1.]
 ..., 
 [ 1.]
 [ 1.]
 [ 1.]]


In [29]:
print(metrics.roc_auc_score(y_test, pred))

0.500322110549


In [111]:
pred = pred.reshape((1, (len(pred))))[0]

In [113]:
pd.Series(pred).unique()

array([ 0.17584203])

In [83]:
sum(pred != 0.17624342)

22983

In [62]:
sum(y_test == 1)

9017